# Deep Convolutional Q-Learning para PacMan

## Part 0 - Instalar paquetes e importar librerias

### Instalar Gymnasium

In [1]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 13.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 77.6 MB/s eta 0:00:00
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446660 sha256=12163d94b25e9c8ff15661af0f4a51e072521291bf1097cdc5eb6cc99e97403c
  Stored in directory: /root/.cache/pip/wheels/6b/1b/ef/a43ff1a2f1736d5711faa1ba4c1f61be1131b8899e6a057811
Successfully built AutoROM.accept-rom-license
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  swig4.0
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:


### Importar las librerias

In [2]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import deque
from torch.utils.data import DataLoader, TensorDataset

## Part 1 - Building the AI

### Crear la arquitectura de la red neuronal

In [3]:
class Network(nn.Module):

  def __init__(self, action_size, seed=42):
    super(Network, self).__init__()
    # Establece una semilla para la reproducibilidad.
    self.seed = torch.manual_seed(seed)

    # Define las capas de la red neuronal convolucional.
    self.conv1 = nn.Conv2d(3, 32, kernel_size=8, stride=4)  # Primera capa convolucional: convierte 3 canales (RGB) en 32 canales, con un kernel de 8x8 y stride de 4.
    self.bn1 = nn.BatchNorm2d(32)  # Normalización por lotes para la primera capa convolucional.

    self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)  # Segunda capa convolucional: convierte 32 canales en 64 canales, con un kernel de 4x4 y stride de 2.
    self.bn2 = nn.BatchNorm2d(64)  # Normalización por lotes para la segunda capa convolucional.

    self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)  # Tercera capa convolucional: mantiene 64 canales con un kernel de 3x3 y stride de 1.
    self.bn3 = nn.BatchNorm2d(64)  # Normalización por lotes para la tercera capa convolucional.

    self.conv4 = nn.Conv2d(64, 128, kernel_size=3, stride=1)  # Cuarta capa convolucional: convierte 64 canales en 128 canales, con un kernel de 3x3 y stride de 1.
    self.bn4 = nn.BatchNorm2d(128)  # Normalización por lotes para la cuarta capa convolucional.

    # Define las capas totalmente conectadas.
    self.fc1 = nn.Linear(10 * 10 * 128, 512)  # Primera capa totalmente conectada: toma la salida de la última capa convolucional (ajustada para tener dimensiones 10x10x128) y la convierte en un vector de 512 dimensiones.
    self.fc2 = nn.Linear(512, 256)  # Segunda capa totalmente conectada: convierte 512 dimensiones a 256 dimensiones.
    self.fc3 = nn.Linear(256, action_size)  # Tercera capa totalmente conectada: convierte 256 dimensiones a `action_size`, el número de acciones posibles.

  def forward(self, state):
    # Define el paso hacia adelante de la red neuronal.

    # Propaga el estado a través de las capas convolucionales y de normalización por lotes con la función de activación ReLU.
    x = F.relu(self.bn1(self.conv1(state)))
    x = F.relu(self.bn2(self.conv2(x)))
    x = F.relu(self.bn3(self.conv3(x)))
    x = F.relu(self.bn4(self.conv4(x)))

    # Aplana la salida de las capas convolucionales para que pueda ser pasada a las capas totalmente conectadas.
    x = x.view(x.size(0), -1)

    # Propaga la salida a través de las capas totalmente conectadas con la función de activación ReLU.
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))

    # Devuelve las salidas de la última capa totalmente conectada, que representa los valores de acción.
    return self.fc3(x)

## Part 2 - Entrenamiento

### Estableciendo el entorno

In [4]:
import gymnasium as gym
env = gym.make('MsPacmanDeterministic-v0', full_action_space = False)
state_shape = env.observation_space.shape
state_size = env.observation_space.shape[0]
number_actions = env.action_space.n
print('State shape: ', state_shape)
print('State size: ', state_size)
print('Number of actions: ', number_actions)

/usr/local/lib/python3.10/dist-packages/gymnasium/envs/registration.py:513: DeprecationWarning: WARN: The environment MsPacmanDeterministic-v0 is out of date. You should consider upgrading to version `v4`.
  logger.deprecation(


State shape:  (210, 160, 3)
State size:  210
Number of actions:  9


### Inicilizar hiperparametros

In [5]:
learning_rate = 5e-4
minibatch_size = 64
discount_factor = 0.99

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Preprocesar los frames

In [6]:
from PIL import Image
from torchvision import transforms

def preprocess_frame(frame):
  # Convierte el array de numpy `frame` a una imagen de PIL.
  frame = Image.fromarray(frame)

  # Define una secuencia de transformaciones a aplicar a la imagen:
  # 1. Resize: Redimensiona la imagen a un tamaño de 128x128 píxeles.
  # 2. ToTensor: Convierte la imagen a un tensor de PyTorch (normaliza los valores de píxeles a [0, 1] y cambia el formato a [C, H, W]).
  preprocess = transforms.Compose([transforms.Resize((128, 128)), transforms.ToTensor()])

  # Aplica las transformaciones definidas a la imagen y añade una dimensión extra en la posición 0 para crear un batch de tamaño 1.
  return preprocess(frame).unsqueeze(0)


### Implementar la DCQN class

In [7]:
class Agent():

  def __init__(self, action_size):
    # Determina el dispositivo de computación (GPU si está disponible, de lo contrario CPU).
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # Número de acciones posibles en el entorno.
    self.action_size = action_size

    # Red neuronal local para estimar los valores Q.
    self.local_qnetwork = Network(action_size).to(self.device)

    # Red neuronal objetivo para estabilizar el entrenamiento.
    self.target_qnetwork = Network(action_size).to(self.device)

    # Optimizador para actualizar los pesos de la red neuronal.
    self.optimizer = optim.Adam(self.local_qnetwork.parameters(), lr=learning_rate)

    # Memoria para almacenar las experiencias de los episodios.
    self.memory = deque(maxlen=10000)

  def step(self, state, action, reward, next_state, done):
    # Preprocesa las imágenes de estado y próximo estado.
    state = preprocess_frame(state)
    next_state = preprocess_frame(next_state)

    # Almacena la experiencia en la memoria.
    self.memory.append((state, action, reward, next_state, done))

    # Si la memoria tiene suficientes experiencias, realiza el aprendizaje.
    if len(self.memory) > minibatch_size:
      experiences = random.sample(self.memory, k=minibatch_size)
      self.learn(experiences, discount_factor)

  def act(self, state, epsilon=0.):
    # Preprocesa el estado.
    state = preprocess_frame(state).to(self.device)

    # Pone la red neuronal en modo evaluación para evitar que se actualicen los pesos durante la inferencia.
    self.local_qnetwork.eval()

    with torch.no_grad():
      # Obtiene los valores Q de la red neuronal local para el estado dado.
      action_values = self.local_qnetwork(state)

    # Restaura la red neuronal al modo de entrenamiento.
    self.local_qnetwork.train()

    # Decisión de acción basada en epsilon-greedy.
    if random.random() > epsilon:
      # Selecciona la acción con el valor Q más alto (explotación).
      return np.argmax(action_values.cpu().data.numpy())
    else:
      # Selecciona una acción aleatoria (exploración).
      return random.choice(np.arange(self.action_size))

  def learn(self, experiences, discount_factor):
    # Extrae las experiencias de la muestra de minibatch.
    states, actions, rewards, next_states, dones = zip(*experiences)

    # Convierte las experiencias en tensores y mueve los tensores al dispositivo de computación.
    states = torch.from_numpy(np.vstack(states)).float().to(self.device)
    actions = torch.from_numpy(np.vstack(actions)).long().to(self.device)
    rewards = torch.from_numpy(np.vstack(rewards)).float().to(self.device)
    next_states = torch.from_numpy(np.vstack(next_states)).float().to(self.device)
    dones = torch.from_numpy(np.vstack(dones).astype(np.uint8)).float().to(self.device)

    # Calcula los valores Q objetivo utilizando la red neuronal objetivo.
    next_q_targets = self.target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1)
    q_targets = rewards + discount_factor * next_q_targets * (1 - dones)

    # Obtiene los valores Q esperados para las acciones tomadas utilizando la red neuronal local.
    q_expected = self.local_qnetwork(states).gather(1, actions)

    # Calcula la pérdida (error cuadrático medio) entre los valores Q esperados y los valores Q objetivos.
    loss = F.mse_loss(q_expected, q_targets)

    # Optimiza la red neuronal.
    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Inicializar el DCQN agent

In [8]:
agent = Agent(number_actions)

### Entrenar el DCQN agent

In [9]:
# Parámetros de entrenamiento
number_episodes = 200  # Número total de episodios para entrenar el agente
maximum_number_timesteps_per_episode = 10000  # Número máximo de pasos por episodio
epsilon_starting_value = 1.0  # Valor inicial de epsilon para la política epsilon-greedy
epsilon_ending_value = 0.01  # Valor final de epsilon
epsilon_decay_value = 0.995  # Tasa de decaimiento de epsilon
epsilon = epsilon_starting_value  # Valor actual de epsilon
scores_on_100_episodes = deque(maxlen=100)  # Cola para almacenar las puntuaciones de los últimos 100 episodios

# Bucle principal de entrenamiento
for episode in range(1, number_episodes + 1):
  # Inicialización al inicio de un nuevo episodio
  state, _ = env.reset()  # Restablece el entorno y obtiene el estado inicial
  score = 0  # Inicializa la puntuación del episodio

  # Bucle para cada paso en el episodio
  for t in range(maximum_number_timesteps_per_episode):
    action = agent.act(state, epsilon)  # Selecciona una acción usando la política epsilon-greedy
    next_state, reward, done, _, _ = env.step(action)  # Ejecuta la acción en el entorno
    agent.step(state, action, reward, next_state, done)  # Almacena la experiencia y actualiza la red
    state = next_state  # Actualiza el estado actual
    score += reward  # Acumula la recompensa del episodio
    if done:
      break  # Termina el episodio si el entorno indica que ha terminado

  # Almacena la puntuación del episodio en la cola de puntuaciones
  scores_on_100_episodes.append(score)

  # Actualiza epsilon (estrategia de exploración)
  epsilon = max(epsilon_ending_value, epsilon_decay_value * epsilon)

  # Imprime el progreso del entrenamiento
  print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)), end="")

  # Imprime información adicional cada 100 episodios
  if episode % 100 == 0:
    print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)))

  # Verifica si se ha resuelto el entorno
  if np.mean(scores_on_100_episodes) >= 500.0:
    print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(episode - 100, np.mean(scores_on_100_episodes)))
    torch.save(agent.local_qnetwork.state_dict(), 'checkpoint.pth')  # Guarda el estado de la red neuronal
    break  # Sale del bucle si se ha resuelto el entorno


Episode 92	Average Score: 304.46

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-0194ce951295>", line 18, in <cell line: 11>
    action = agent.act(state, epsilon)  # Selecciona una acción usando la política epsilon-greedy
  File "<ipython-input-7-19f11da9187a>", line 37, in act
    state = preprocess_frame(state).to(self.device)
  File "<ipython-input-6-87306cb65438>", line 14, in preprocess_frame
    return preprocess(frame).unsqueeze(0)
  File "/usr/local/lib/python3.10/dist-packages/torchvision/transforms/transforms.py", line 95, in __call__
    img = t(img)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1532, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1541, in _call_impl
    return forward_call(*args, **kwa

TypeError: object of type 'NoneType' has no len()

## Part 3 - Ver resultados

In [ ]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display
from gym.wrappers.monitoring.video_recorder import VideoRecorder

# Crear el video
def show_video_of_model(agent, env_name):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action)
    env.close()
    imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent, 'MsPacmanDeterministic-v0')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()